In [ ]:
# 모델 학습과 검증 및 투자 예측 예시

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 결측값 처리 (FutureWarning 해결)
df.ffill(inplace=True)

# 타겟 설정: 다음 날 종가가 오늘보다 높으면 1, 낮으면 0
df['Target'] = (df['종가'].shift(-1) > df['종가']).astype(int)

# 특성 선택 (이동 평균, RSI, 볼린저 밴드)
features = ['MA_20', 'RSI_14', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']

# X: 특성 데이터셋, y: 타겟
X = df[features]
y = df['Target']

# 데이터셋을 훈련과 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습 (랜덤 포레스트 분류기)
classifier = RandomForestClassifier(n_estimators=500, random_state=42)
classifier.fit(X_train, y_train)

# 예측
y_pred = classifier.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("분류 모델 정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)


분류 모델 정확도: 0.5141700404858299
혼동 행렬:
 [[68 48]
 [72 59]]


In [ ]:
'''
분류모델은 정확도 상당히 낮음...
거의 랜덤 확률
'''

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
import pandas as pd

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 날짜 열이 있다면 제거 또는 타임스탬프로 변환
if '날짜' in df.columns:
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].map(pd.Timestamp.toordinal)  # 타임스탬프로 변환

# 타겟 설정 (다음 날의 종가)
y_reg = df['종가'].shift(-1)

# 특성에서 NaN 값을 가진 행 제거 (X와 y_reg에 모두 적용)
df.dropna(inplace=True)

# 특성(X)와 타겟(y) 설정 (예시로 특성을 df에서 '종가'를 제외한 나머지로 설정)
X = df.drop(columns=['종가'])  # 특성은 '종가'를 제외한 다른 모든 열로 설정
y_reg = df['종가'].shift(-1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# y_train에 남아 있을 수 있는 NaN 값을 처리 (평균으로 대체)
imputer = SimpleImputer(strategy='mean')
y_train = imputer.fit_transform(y_train.values.reshape(-1, 1)).ravel()

# 하이퍼파라미터 검색을 위한 랜덤 포레스트 회귀 모델 설정
regressor = RandomForestRegressor(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [200, 500, 800],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]  # 'auto' 대신 'sqrt'나 'log2' 사용
}

# GridSearchCV를 사용한 하이퍼파라미터 최적화 설정
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=2)

# 모델 학습 (최적 하이퍼파라미터 찾기)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)

# 최적의 모델로 예측 수행
best_regressor = grid_search.best_estimator_
y_pred_reg = best_regressor.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred_reg)
r2 = r2_score(y_test, y_pred_reg)

print("최적화된 회귀 모델 MAE:", mae)
print("최적화된 회귀 모델 R²:", r2)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  13.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  13.6s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  15.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  15.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  20.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=  32.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=  34.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=

In [ ]:
# RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 날짜 열이 있다면 제거 또는 타임스탬프로 변환
if '날짜' in df.columns:
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].map(pd.Timestamp.toordinal)  # 타임스탬프로 변환

# 타겟 설정 (다음 날의 종가)
y_reg = df['종가'].shift(-1)

# 특성에서 NaN 값을 가진 행 제거 (X와 y_reg에 모두 적용)
df.dropna(inplace=True)

# 특성(X)와 타겟(y) 설정 (예시로 특성을 df에서 '종가'를 제외한 나머지로 설정)
X = df.drop(columns=['종가'])  # 특성은 '종가'를 제외한 다른 모든 열로 설정
y_reg = df['종가'].shift(-1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# y_train에 남아 있을 수 있는 NaN 값을 처리 (평균으로 대체)
imputer = SimpleImputer(strategy='mean')
y_train = imputer.fit_transform(y_train.values.reshape(-1, 1)).ravel()

# 최적의 하이퍼파라미터를 이용한 랜덤 포레스트 회귀 모델 설정
best_regressor = RandomForestRegressor(
    n_estimators=800,
    max_depth=10,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

# 모델 학습
best_regressor.fit(X_train, y_train)

# 예측 수행
y_pred_reg = best_regressor.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred_reg)
r2 = r2_score(y_test, y_pred_reg)

print("최적화된 회귀 모델 MAE:", mae)
print("최적화된 회귀 모델 R²:", r2)


최적화된 회귀 모델 MAE: 1.3192841470498364
최적화된 회귀 모델 R²: 0.992598277660945


In [ ]:
'''
회귀 모델의 성능 결과를 보면, MAE (Mean Absolute Error) 값이 1.449이고, R² 값이 0.989로 매우 높은 설명력을 보여준다.
이는 모델이 테스트 데이터에서 실제 종가와 매우 근접한 예측을 했다는 것을 의미한다.

MAE (1.449): 평균적으로 예측 값과 실제 값의 차이가 약 1.449 정도라는 의미. 
종가 단위에서 이 정도의 오차는 상당히 작은 편으로, 모델이 정확하게 예측하고 있음을 시사한다.

R² (0.989): R² 값이 1에 가까울수록 모델이 데이터를 잘 설명하고 있다는 의미. 
0.989는 매우 높은 값으로, 입력된 특성들이 다음 날의 종가를 예측하는 데 크게 기여하고 있다는 것을 나타낸다.
이 결과는 현재 사용 중인 랜덤 포레스트 회귀 모델이 매우 잘 작동하고 있다는 것을 보여주며, 
추가적인 개선 없이도 이미 상당히 좋은 성과를 얻고 있음을 시사한다. 
만약 더 나은 결과를 원한다면, 하이퍼파라미터 튜닝 또는 추가적인 특성 공학을 시도해볼 수 있다.
'''

In [ ]:
# Gradient Boosting Model

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 날짜 열이 있다면 제거 또는 타임스탬프로 변환
if '날짜' in df.columns:
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].map(pd.Timestamp.toordinal)  # 타임스탬프로 변환

# 타겟 설정 (다음 날의 종가)
y_reg = df['종가'].shift(-1)

# 특성에서 NaN 값을 가진 행 제거 (X와 y_reg에 모두 적용)
df.dropna(inplace=True)

# 특성(X)와 타겟(y) 설정 (예시로 특성을 df에서 '종가'를 제외한 나머지로 설정)
X = df.drop(columns=['종가'])  # 특성은 '종가'를 제외한 다른 모든 열로 설정
y_reg = df['종가'].shift(-1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# y_train에 남아 있을 수 있는 NaN 값을 처리 (평균으로 대체)
imputer = SimpleImputer(strategy='mean')
y_train = imputer.fit_transform(y_train.values.reshape(-1, 1)).ravel()

# XGBoost 회귀 모델 설정
best_regressor = XGBRegressor(
    n_estimators=800,
    max_depth=10,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 모델 학습
best_regressor.fit(X_train, y_train)

# 예측 수행
y_pred_reg = best_regressor.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred_reg)
r2 = r2_score(y_test, y_pred_reg)

print("XGBoost 회귀 모델 MAE:", mae)
print("XGBoost 회귀 모델 R²:", r2)


XGBoost 회귀 모델 MAE: 1.3834646600165972
XGBoost 회귀 모델 R²: 0.991687106466816


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 날짜 열이 있다면 제거 또는 타임스탬프로 변환
if '날짜' in df.columns:
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['날짜'] = df['날짜'].map(pd.Timestamp.toordinal)  # 타임스탬프로 변환

# 타겟 설정 (다음 날의 종가)
df['y_reg'] = df['종가'].shift(-1)

# 특성에서 NaN 값을 가진 행 제거 (X와 y_reg에 모두 적용)
df.dropna(inplace=True)

# 특성(X)와 타겟(y) 설정 (예시로 특성을 df에서 '종가'를 제외한 나머지로 설정)
X = df.drop(columns=['종가', 'y_reg'])  # 특성은 '종가'와 타겟('y_reg')을 제외한 다른 모든 열로 설정
y_reg = df['y_reg']  # 타겟 값으로 y_reg 설정

# 데이터 정규화 (LSTM은 정규화된 데이터로 학습하는 것이 일반적)
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y_reg.values.reshape(-1, 1))

# 데이터를 시계열에 맞는 형태로 변환 (LSTM 입력 형식: [samples, timesteps, features])
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# LSTM 모델 구성
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32)

# 예측 수행
y_pred_scaled = model.predict(X_test)

# 스케일을 원래대로 복구
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test)

# 평가
mae = mean_absolute_error(y_test_actual, y_pred)
r2 = r2_score(y_test_actual, y_pred)

print("LSTM 회귀 모델 MAE:", mae)
print("LSTM 회귀 모델 R²:", r2)


/home/yim/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.2058
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0132
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0047
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0018
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 8.4826e-04
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7.6896e-04
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 5.5956e-04
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 5.4233e-04
Epoch 9/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4.9844e-04
Epoch 10/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.2356e-04
Epoch 11/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.1315e-04
Epoch 12/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 5.5303e-04
Epoch 13/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 4.3159e-04
Epoch 14/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 4.9490e-04
Epoch 15/100
31/31 

In [ ]:
# RandomForest 모델이 가장 최적화된 모델임을 알 수 있음

In [ ]:
# 투자 예시 (하루 기준)

In [ ]:
import pandas as pd

# 데이터 로드
df = pd.read_csv('store_data/process/OHLCV_info/calculate_total_year/건설업.csv')

# 날짜 열이 있다면 변환
if '날짜' in df.columns:
    df['날짜'] = pd.to_datetime(df['날짜'])

# 매수/매도 신호를 결정하는 함수 정의
def trading_signal(row):
    # 이동평균 규칙
    if row['종가'] > row['MA_20']:
        ma_signal = 1  # 매수 신호
    elif row['종가'] < row['MA_20']:
        ma_signal = -1  # 매도 신호
    else:
        ma_signal = 0   # 신호 없음

    # RSI 규칙
    if row['RSI_14'] < 30:
        rsi_signal = 1  # 매수 신호
    elif row['RSI_14'] > 70:
        rsi_signal = -1  # 매도 신호
    else:
        rsi_signal = 0   # 신호 없음

    # 볼린저 밴드 규칙
    if row['종가'] < row['BBL_20_2.0']:
        bb_signal = 1  # 매수 신호
    elif row['종가'] > row['BBU_20_2.0']:
        bb_signal = -1  # 매도 신호
    else:
        bb_signal = 0   # 신호 없음

    # 종합 신호 (여기서는 단순 합계로 매수/매도 결정)
    total_signal = ma_signal + rsi_signal + bb_signal
    
    if total_signal > 0:
        return '매수'  # 매수 신호
    elif total_signal < 0:
        return '매도'  # 매도 신호
    else:
        return '유지'  # 신호 없음

# 각 행에 대해 매수/매도 신호 생성
df['Signal'] = df.apply(trading_signal, axis=1)

# 매수/매도 신호를 확인
print(df[['날짜', '종가', 'MA_20', 'RSI_14', 'BBL_20_2.0', 'BBU_20_2.0', 'Signal']].tail(20))

             날짜     종가    MA_20     RSI_14  BBL_20_2.0  BBU_20_2.0 Signal
1194 2023-11-30  75.19  73.3340  63.530013   69.309558   77.358442     매수
1195 2023-12-01  75.14  73.6020  63.143344   69.855671   77.348329     매수
1196 2023-12-04  74.81  73.7650  60.525016   70.107348   77.422652     매수
1197 2023-12-05  74.37  73.9205  57.123779   70.442009   77.398991     매수
1198 2023-12-06  75.59  74.1370  63.284642   70.812214   77.461786     매수
1199 2023-12-07  75.00  74.3095  58.878498   71.187703   77.431297     매수
1200 2023-12-08  74.72  74.4855  56.855374   71.706720   77.264280     매수
1201 2023-12-11  74.30  74.6650  53.865503   72.485840   76.844160     매도
1202 2023-12-12  74.75  74.8385  56.504697   73.309405   76.367595     매도
1203 2023-12-13  73.48  74.8480  48.135450   73.355585   76.340415     매도
1204 2023-12-14  73.66  74.8585  49.282063   73.403079   76.313921     매도
1205 2023-12-15  73.89  74.8600  50.779446   73.408642   76.311358     매도
1206 2023-12-18  72.99  74.8055  45.16